## There are some instructions you need to follow:
<li> You only need to write your code in the comment area "Your Code Here".</li>
<li>Do not upload your own file. Please make the necessary changes in the Jupyter notebook file already present in the server.</li>
<li>Please note, there are several cells in the Assignment Jupyter notebook that are empty and read only. Do not attempt to remove them or   edit them. They are used in grading your notebook. Doing so might lead to 0 points.</li>

In [4]:
import nltk
import os
import _sqlite3
from nltk.corpus import PlaintextCorpusReader
from nltk import sent_tokenize,word_tokenize 
from gensim import corpora, models, similarities
from gensim.models.ldamodel import LdaModel
from gensim.parsing.preprocessing import STOPWORDS
from gensim.similarities.docsim import Similarity

# Question 1

In [22]:
"""
Question 1

Write a function that takes the file name of the Wikipedia page containing all Greek ancient
philosophers (saved as "Index.html" in your workspace) and returns a list tuples containing 
the name of the philosopher and the path to its individual article file.

Example of use: get_philosophers("Index.html")

The output should be a list of tuples:

[('Acrion', 'Philosophers/Acrion.html'),
 ('Adrastus of Aphrodisias', 'Philosophers/Adrastus of Aphrodisias.html'),
 ('Aedesia', 'Philosophers/Aedesia.html'),
 ('Aedesius', 'Philosophers/Aedesius.html'),
 ('Aeneas of Gaza', 'Philosophers/Aeneas of Gaza.html'),
 ('Aenesidemus', 'Philosophers/Aenesidemus.html'),
 ...]
 
  
NOTE: For processing speed purposes, the table in "Index.html" has been shortened compared
to the one online on wikipedia.org. Do not worry if you do not find some philosophers in 
your results, this is made on purpose. 

"""

def get_philosophers(filename):
    
    import codecs
    from bs4 import BeautifulSoup
    f = codecs.open(filename, 'r', 'utf-8')
    soup = BeautifulSoup(f.read(),'lxml')
    
    philosopher_list = []
    
    tableRows = soup.find_all('table',class_='wikitable sortable')[0]
    data_rows = tableRows.find_all('tr')[1:]
    philosopher_name = [] #list containing each philosopher's name
    # grabs philosopher name from title an appends to list
    for data in data_rows:
        philosopher_name.append(data.find('a').get('title'))

    # adding path and filename at the end of philosopher's name
    for name in philosopher_name:
        x = name
        y = name,"Philosophers/"+ x + '.html'
        philosopher_list.append(y)

#   returns philosopher list as tuple
#     philosopher_tuple_list = tuple(philosopher_list)
    return philosopher_list

# Once done, try this:
# get_philosophers("Index.html")

# Question 1: Non-Function

In [23]:
import codecs
from bs4 import BeautifulSoup
f = codecs.open("Index.html", 'r', 'utf-8')
soup = BeautifulSoup(f.read(),'lxml')

# reference: http://savvastjortjoglou.com/nba-draft-part01-scraping.html
# tuples of philosopher name w/ path 
philosopher_list = []

# creates list of philosopher name
tableRows = soup.find_all('table',class_='wikitable sortable')[0]
data_rows = tableRows.find_all('tr')[1:]
philosopher_name = [] #list containing each philosopher's name

# grabs philosopher name from title and appends to list
for data in data_rows:
    philosopher_name.append(data.find('a').get('title'))
#     print(type(data))


# # loops through a list indices 
# for i in range(len(data_rows)):
#     # for each table data element from each table row
#     td = data_rows[i].find_all('td')[0].get_text().strip()
# #     philosopher_name.append(td)


# adding path and filename at the end of philosopher's name
for name in philosopher_name:
    x = name
    y = name,"Philosophers/"+ x + '.html'
    philosopher_list.append(y)

# philosopher_list

# Question 2

In [24]:
"""
Question 2


Write a function that scrapes the text on a philosophers’s page and returns it as a text 
string. The input is the name of the file that contains the philosopher's page.

Example of use: get_text('Philosophers/Acrion.html')
should output the text of the page.
'Acrion was a Locrian and a Pythagorean philosopher...'
"""

def get_text(file):
    import codecs
    from bs4 import BeautifulSoup
    f = codecs.open(file, 'r', 'utf-8')
    page_soup = BeautifulSoup(f.read(),'lxml')

    all_text = ""

    for tag in page_soup.find_all('p'):
        all_text += tag.get_text()
    return all_text
# Once done, try this:
# get_text("Philosophers/Acrion.html")

# Question 2: Non-Function

In [25]:
# looks at page index.html (all philosophers)
filename = "Philosophers/Acrion.html"

import codecs
from bs4 import BeautifulSoup
f = codecs.open(filename, 'r', 'utf-8')
page_soup = BeautifulSoup(f.read(),'lxml')


all_text = ""

# takes the 2nd index from list philosopher_list
for philosopher_filepath in philosopher_list:
    philosopher_filepath = philosopher_filepath[1]
    
    filepath = codecs.open(philosopher_filepath, 'r', 'utf-8')
    page_soup = BeautifulSoup(filepath.read(),'lxml')
    
    for tag in page_soup.find_all('p'):
        all_text += tag.get_text()
# all_text

# Question 3 (functional)

In [26]:
"""
Question 3

Use the files under "Philosophers" folder to construct an LSI model.
Then, use the LSI model to find the most similar philosopher for each of the philosophers
found in Question 1, based on the content of their Wikipedia articles. You should not go
online to scrape the data; everything you need is in your Jupyter notebook working directory.

The function should have as input the list of tuples created in Question 1.

The output format should be a list of tuples too. Each tuple should contain a philosopher's name
and its most similar other philosopher. Please note both names can't be the same.

The output should look like that:

[('Acrion', 'Arignote'),
 ('Adrastus of Aphrodisias', 'Lycophron (Sophist)'),
 ('Aedesia', 'Heliodorus of Alexandria'),
 ('Aedesius', 'Chrysanthius'),
 ('Aeneas of Gaza', 'Archytas'),
 ...]


"""

# def run1(filenames):
# ###
# ### YOUR CODE HERE
# ###

# # Once done, try this:
# run(filenames)

'\nQuestion 3\n\nUse the files under "Philosophers" folder to construct an LSI model.\nThen, use the LSI model to find the most similar philosopher for each of the philosophers\nfound in Question 1, based on the content of their Wikipedia articles. You should not go\nonline to scrape the data; everything you need is in your Jupyter notebook working directory.\n\nThe function should have as input the list of tuples created in Question 1.\n\nThe output format should be a list of tuples too. Each tuple should contain a philosopher\'s name\nand its most similar other philosopher. Please note both names can\'t be the same.\n\nThe output should look like that:\n\n[(\'Acrion\', \'Arignote\'),\n (\'Adrastus of Aphrodisias\', \'Lycophron (Sophist)\'),\n (\'Aedesia\', \'Heliodorus of Alexandria\'),\n (\'Aedesius\', \'Chrysanthius\'),\n (\'Aeneas of Gaza\', \'Archytas\'),\n ...]\n\n\n'

# Question 3 (non-functional)

In [27]:
import nltk
import os
import _sqlite3
from nltk.corpus import PlaintextCorpusReader
from nltk import sent_tokenize,word_tokenize 
from gensim import corpora, models, similarities
from gensim.models.ldamodel import LdaModel
from gensim.parsing.preprocessing import STOPWORDS
from gensim.similarities.docsim import Similarity

In [36]:
# text preprocessing
# all_text = scraped description of all philosophers
striptext = all_text.replace('\n\n', ' ')
striptext = striptext.replace('\n', ' ')
sentences = sent_tokenize(striptext)
texts = [[word for word in sentence.lower().split()
         if word not in STOPWORDS and word.isalnum()]
         for sentence in sentences ]
len(texts)

2473

In [45]:
# create word dictionary for each word in a text 
dictionary = corpora.Dictionary(texts)

# create corpus: assigns ints to words
# https://radimrehurek.com/gensim/corpora/dictionary.html#gensim.corpora.dictionary.Dictionary.doc2bow
# (token_id,token_count)
corpus = [dictionary.doc2bow(text) for text in texts]

In [ ]:
# create LSI model


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###


In [ ]:
###
### AUTOGRADER TEST - DO NOT REMOVE
###
